# Client demo for the Kimina Lean Server

This notebook shows how to use the Kimina Lean Server for three different tasks:
1. Large-scale verification: send a large batch of Lean codes to the server and get verification feedbacks.
2. Proof data extraction: extract all tactics and tactic states from a Lean code.
3. Interaction: send code, receive verification and extracted data, update the code, and repeat.

For all three tasks, you need to launch the Kimina Lean Server first. You can do this by running the following command in your terminal:
```bash
python -m server
```

## 1. Large-scale verification

This first section demonstrates how to use the Kimina Lean Server for large-scale verification.

### 1.1. Import libraries

In [ ]:
import sys
import nest_asyncio
import uuid

sys.path.insert(0, "YOUR_PATH_TO_kimina-lean-server")

from client.client import Lean4Client
from client.infotree import extract_data
from utils.proof_utils import split_proof_header, parse_client_response

nest_asyncio.apply()

### 1.2. Initialize a client

In [2]:
client = Lean4Client(base_url="http://127.0.0.1:12332")

### 1.3. Load Lean codes

Let's load Lean codes that we will verify. For this demo, we will use the Lean Workbook dataset.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Goedel-LM/Lean-workbook-proofs", split="train")
dataset = dataset.shuffle(seed=42)
dataset = dataset.select(range(1000))

In [4]:
codes = [
    {"custom_id": sample["problem_id"], "proof": sample["full_proof"]}
    for sample in dataset
]

### 1.4. Send codes to the server

In [5]:
response = client.verify(codes, timeout=60)

In [6]:
verification_results = [
    parse_client_response(item)
    for item in response["results"]
]

We can now have access to the verification result of any proof. For example, here is the verification result of the first proof in the dataset.

In [7]:
print(verification_results[0])

{'has_error': False, 'is_valid_no_sorry': True, 'is_valid_with_sorry': True}


As we can see, this proof doesn't have any errors, is valid and doesn't use `sorry`.

Let's check the percentage of proofs that are valid without using `sorry`.

In [8]:
count_valid_no_sorry = sum(r["is_valid_no_sorry"] for r in verification_results)

valid_rate = count_valid_no_sorry / len(verification_results)
print(f"Valid rate: {valid_rate:.2%}")

Valid rate: 97.50%


Not 100% of the proofs are valid, for two reasons: 
1. These proofs were originally written with Lean 4.9.0, and our server is using Lean 4.15.0.
2. We are using a timeout of 60 seconds for the verification. The timeout can be set to a higher value if needed.

## 2. Data extraction

This second section demonstrates how to use the Kimina Lean Server to extract proof data (tactics and tactic states) from a Lean code.

### 2.1. Import libraries

In [ ]:
import sys
import nest_asyncio
import uuid

sys.path.insert(0, "YOUR_PATH_TO_kimina-lean-server")

from client.client import Lean4Client
from client.infotree import extract_data
from utils.proof_utils import split_proof_header

nest_asyncio.apply()

### 2.2. Initialize a client

In [10]:
client = Lean4Client(base_url="http://127.0.0.1:12332")

### 2.3. Load the Lean code

Let's load the Lean code from a file. The code should contain the imports, a statement and its proof.

In [11]:
lean_file = "demo_extraction.lean"
with open(lean_file, "r") as f:
    lean_code = f.read()

For this demo, we will extract tactics and tactic states from the following proof from Lean Workbook.

In [12]:
print(lean_code)

import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/- Let $a,b,c>0$ . Prove that: $\frac{a^{2}}{b+c}+\frac{b^{2}}{a+c}+\frac{16c^{2}}{a+b}\geq \frac{1}{9}(64c-a-b)$ -/
theorem lean_workbook_10058 (a b c : ℝ) (ha : 0 < a) (hb : 0 < b) (hc : 0 < c) : (a^2 / (b + c) + b^2 / (a + c) + 16 * c^2 / (a + b)) ≥ (1 / 9) * (64 * c - a - b)  := by
  have h₁ : 0 < a + b + c := by linarith
  have h₂ : 0 < a * b := by positivity
  have h₃ : 0 < a * c := by positivity
  have h₄ : 0 < b * c := by positivity
  field_simp [h₁.ne', h₂.ne', h₃.ne', h₄.ne']
  rw [div_le_div_iff]
  nlinarith [sq_nonneg (a - b), sq_nonneg (a - 4 * c), sq_nonneg (b - 4 * c), sq_nonneg (a + b - 4 * c),
    sq_nonneg (a + b - 2 * c), sq_nonneg (a + b + 2 * c)]
  all_goals nlinarith


### 2.4. Send the code to the Server and get a response

In [13]:
codes = [{"proof": lean_code, "custom_id": str(uuid.uuid4())}]

response = client.verify(codes, timeout=10, infotree_type="original")

### 2.5. Extract the data (tactics and tactic states) from the infotree

First, get the infotree from the response.

In [14]:
infotree = response['results'][0]['response']['infotree']

Let's first split the Lean code into header (imports) and body (statement and proof). The positions in the infotree returned by the server correspond to the positions in the body (not the full code), which is why we first have to retrieve the body in order to extract data from the infotree.

Then, we can extract the tactics and tactic states from the infotree.

In [15]:
header, body = split_proof_header(lean_code)

intervals = extract_data(infotree, body)

We can now print the results.

In [16]:
for interval in intervals:
    print("goalsBefore:")
    print("\n".join(goal for goal in interval["goalsBefore"]))
    print("-" * 3)
    print("tactic:")
    print(interval["tactic"])
    print("-" * 3)
    print("goalsAfter:")
    print("\n".join(goal for goal in interval["goalsAfter"]))
    print("-" * 20)

goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
⊢ a ^ 2 / (b + c) + b ^ 2 / (a + c) + 16 * c ^ 2 / (a + b) ≥ 1 / 9 * (64 * c - a - b)
---
tactic:
by
  have h₁ : 0 < a + b + c :=
---
goalsAfter:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
⊢ 0 < a + b + c
--------------------
goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
⊢ 0 < a + b + c
---
tactic:
 by linarith
---
goalsAfter:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ a ^ 2 / (b + c) + b ^ 2 / (a + c) + 16 * c ^ 2 / (a + b) ≥ 1 / 9 * (64 * c - a - b)
--------------------
goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ a ^ 2 / (b + c) + b ^ 2 / (a + c) + 16 * c ^ 2 / (a + b) ≥ 1 / 9 * (64 * c - a - b)
---
tactic:

  have h₂ : 0 < a * b :=
---
goalsAfter:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ 0 < a * b
--------------------
goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ 0 < a * b
---
tactic:
 by positivity
---
goalsAft

## 3. Interaction

This third section demonstrates how to use the Kimina Lean Server to interact with Lean 4 and obtain proof data (tactics and tactic states) from the server. This can be useful when implementing a tree search algorithm with a proof completion model.

### 3.1. Import libraries

In [ ]:
import sys
import nest_asyncio
import uuid

sys.path.insert(0, "YOUR_PATH_TO_kimina-lean-server")

from client.client import Lean4Client
from client.infotree import extract_data
from utils.proof_utils import split_proof_header, parse_client_response

nest_asyncio.apply()

### 3.2. Initialize a client

In [18]:
client = Lean4Client(base_url="http://127.0.0.1:12332")

### 3.3. Load the statement

Let's load the Lean code from a file. The code should contain the imports, a statement and a `sorry`.

In [19]:
lean_file = "demo_search_1.lean"
with open(lean_file, "r") as f:
    lean_code = f.read()

For this demo, we will use the following statement from Lean Workbook.

In [20]:
print(lean_code)

import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/- Let $a,b,c>0$ . Prove that: $\frac{a^{2}}{b+c}+\frac{b^{2}}{a+c}+\frac{16c^{2}}{a+b}\geq \frac{1}{9}(64c-a-b)$ -/
theorem lean_workbook_10058 (a b c : ℝ) (ha : 0 < a) (hb : 0 < b) (hc : 0 < c) : (a^2 / (b + c) + b^2 / (a + c) + 16 * c^2 / (a + b)) ≥ (1 / 9) * (64 * c - a - b)  := by
  sorry


### 3.4. Send the code to the server and get a response

In [21]:
codes = [{"proof": lean_code, "custom_id": str(uuid.uuid4())}]

response = client.verify(codes, timeout=10, infotree_type="original")

### 3.5. Extract the initial tactic state from the infotree

This is the exact same process as in the data extraction example.

In [22]:
infotree = response['results'][0]['response']['infotree']

In [23]:
header, body = split_proof_header(lean_code)

intervals = extract_data(infotree, body)

In [24]:
for interval in intervals:
    print("goalsBefore:")
    print("\n".join(goal for goal in interval["goalsBefore"]))
    print("-" * 3)
    print("tactic:")
    print(interval["tactic"])
    print("-" * 3)
    print("goalsAfter:")
    print("\n".join(goal for goal in interval["goalsAfter"]))
    print("-" * 20)

goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
⊢ a ^ 2 / (b + c) + b ^ 2 / (a + c) + 16 * c ^ 2 / (a + b) ≥ 1 / 9 * (64 * c - a - b)
---
tactic:
by
  sorry
---
goalsAfter:

--------------------


We now have access to the initial tactic state, which can be used with the statement as input for a proof completion model.

### 3.6. Send a first proof attempt to the server

We'll send a first proof attempt to the server. This proof attempt can be generated by a language model, prompted with the theorem statement and the initial tactic state. 

For this demo, let's load a proof attempt from a file.

In [25]:
lean_file = "demo_search_2.lean"
with open(lean_file, "r") as f:
    attempt_1 = f.read()

Here is our first proof attempt.

In [26]:
print(attempt_1)

import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/- Let $a,b,c>0$ . Prove that: $\frac{a^{2}}{b+c}+\frac{b^{2}}{a+c}+\frac{16c^{2}}{a+b}\geq \frac{1}{9}(64c-a-b)$ -/
theorem lean_workbook_10058 (a b c : ℝ) (ha : 0 < a) (hb : 0 < b) (hc : 0 < c) : (a^2 / (b + c) + b^2 / (a + c) + 16 * c^2 / (a + b)) ≥ (1 / 9) * (64 * c - a - b)  := by
  have h₁ : 0 < a + b + c := by linarith
  have h₂ : 0 < a * b := by positivity
  have h₃ : 0 < a * c := by positivity
  have h₄ : 0 < b * c := by positivity
  field_simp [h₁.ne', h₂.ne', h₃.ne', h₄.ne']
  rw [div_le_div_iff]
  nlinarith [ha, hb, hc]


Let's send it to the server.

In [27]:
codes = [{"proof": attempt_1, "custom_id": str(uuid.uuid4())}]

response = client.verify(codes, timeout=10, infotree_type="original")

And let's now have a look at the response. First, we can check the verification results.

In [28]:
parse_client_response(response['results'][0])

{'has_error': True, 'is_valid_no_sorry': False, 'is_valid_with_sorry': False}

As we can see, the proof is not valid. Let's check the messages.

In [29]:
messages = response['results'][0]['response']['messages']

print(messages)

[{'severity': 'warning', 'pos': {'line': 13, 'column': 6}, 'endPos': {'line': 13, 'column': 20}, 'data': '`div_le_div_iff` has been deprecated: use `div_le_div_iff₀` instead'}, {'severity': 'error', 'pos': {'line': 14, 'column': 2}, 'endPos': {'line': 14, 'column': 24}, 'data': 'linarith failed to find a contradiction\ncase a\na b c : ℝ\nha : 0 < a\nhb : 0 < b\nhc : 0 < c\nh₁ : 0 < a + b + c\nh₂ : 0 < a * b\nh₃ : 0 < a * c\nh₄ : 0 < b * c\na✝ :\n  (64 * c - a - b) * ((b + c) * (a + c) * (a + b)) >\n    ((a ^ 2 * (a + c) + b ^ 2 * (b + c)) * (a + b) + 16 * c ^ 2 * ((b + c) * (a + c))) * 9\n⊢ False failed'}]


We have two messages:
- The first one is a warning message, indicating that `div_le_div_iff` has been deprecated. It is not an error message.
- The second one is an error message: it indicates that the `linarith` tactic from line 21 failed.

Let's have a look at the tactics and tactic states extracted from the infotree.

In [30]:
infotree = response['results'][0]['response']['infotree']

header, body = split_proof_header(attempt_1)

intervals = extract_data(infotree, body)

In [31]:
for interval in intervals:
    print("goalsBefore:")
    print("\n".join(goal for goal in interval["goalsBefore"]))
    print("-" * 3)
    print("tactic:")
    print(interval["tactic"])
    print("-" * 3)
    print("goalsAfter:")
    print("\n".join(goal for goal in interval["goalsAfter"]))
    print("-" * 20)

goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
⊢ a ^ 2 / (b + c) + b ^ 2 / (a + c) + 16 * c ^ 2 / (a + b) ≥ 1 / 9 * (64 * c - a - b)
---
tactic:
by
  have h₁ : 0 < a + b + c :=
---
goalsAfter:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
⊢ 0 < a + b + c
--------------------
goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
⊢ 0 < a + b + c
---
tactic:
 by linarith
---
goalsAfter:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ a ^ 2 / (b + c) + b ^ 2 / (a + c) + 16 * c ^ 2 / (a + b) ≥ 1 / 9 * (64 * c - a - b)
--------------------
goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ a ^ 2 / (b + c) + b ^ 2 / (a + c) + 16 * c ^ 2 / (a + b) ≥ 1 / 9 * (64 * c - a - b)
---
tactic:

  have h₂ : 0 < a * b :=
---
goalsAfter:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ 0 < a * b
--------------------
goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ 0 < a * b
---
tactic:
 by positivity
---
goalsAft

### 3.7. Send a new proof attempt

We can reconstruct the proof attempt from the extracted tactics, stopping right before the failed tactic. From the message, we know that the last tactic failed. The correct tactics can be concatenated to give the start of our proof attempt.

In [32]:
current_proof = "".join(intervals[i]["tactic"] for i in range(len(intervals) - 1))

print(current_proof)

by
  have h₁ : 0 < a + b + c := by linarith
  have h₂ : 0 < a * b := by positivity
  have h₃ : 0 < a * c := by positivity
  have h₄ : 0 < b * c := by positivity
  field_simp [h₁.ne', h₂.ne', h₃.ne', h₄.ne']
  rw [div_le_div_iff]


We can also have access to the last tactic state before the failed tactic.

In [33]:
print("\n".join(goal for goal in intervals[-1]["goalsBefore"]))

a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
h₂ : 0 < a * b
h₃ : 0 < a * c
h₄ : 0 < b * c
⊢ (64 * c - a - b) * ((b + c) * (a + c) * (a + b)) ≤
    ((a ^ 2 * (a + c) + b ^ 2 * (b + c)) * (a + b) + 16 * c ^ 2 * ((b + c) * (a + c))) * 9
case b0
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
h₂ : 0 < a * b
h₃ : 0 < a * c
h₄ : 0 < b * c
⊢ 0 < 9
case d0
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
h₂ : 0 < a * b
h₃ : 0 < a * c
h₄ : 0 < b * c
⊢ 0 < (b + c) * (a + c) * (a + b)


We can now generate a new proof attempt. This proof attempt can be generated by a language model, prompted with the theorem statement, the current proof, and the current tactic state. 

For this demo, let's load a new proof attempt from a file.

In [34]:
lean_file = "demo_search_3.lean"
with open(lean_file, "r") as f:
    attempt_2 = f.read()

In [35]:
print(attempt_2)

import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/- Let $a,b,c>0$ . Prove that: $\frac{a^{2}}{b+c}+\frac{b^{2}}{a+c}+\frac{16c^{2}}{a+b}\geq \frac{1}{9}(64c-a-b)$ -/
theorem lean_workbook_10058 (a b c : ℝ) (ha : 0 < a) (hb : 0 < b) (hc : 0 < c) : (a^2 / (b + c) + b^2 / (a + c) + 16 * c^2 / (a + b)) ≥ (1 / 9) * (64 * c - a - b)  := by
  have h₁ : 0 < a + b + c := by linarith
  have h₂ : 0 < a * b := by positivity
  have h₃ : 0 < a * c := by positivity
  have h₄ : 0 < b * c := by positivity
  field_simp [h₁.ne', h₂.ne', h₃.ne', h₄.ne']
  rw [div_le_div_iff]
  nlinarith [sq_nonneg (a - b), sq_nonneg (a - 4 * c), sq_nonneg (b - 4 * c), sq_nonneg (a + b - 4 * c),
    sq_nonneg (a + b - 2 * c), sq_nonneg (a + b + 2 * c)]
  all_goals nlinarith



Let's do the same as before: send it to the server and have a look at the response.

In [36]:
codes = [{"proof": attempt_2, "custom_id": str(uuid.uuid4())}]

response = client.verify(codes, timeout=10, infotree_type="original")

infotree = response['results'][0]['response']['infotree']

In [37]:
parse_client_response(response['results'][0])

{'has_error': False, 'is_valid_no_sorry': True, 'is_valid_with_sorry': True}

This time, the proof seems correct!

Let's have a look at the extracted tactics and tactic states.

In [38]:
header, body = split_proof_header(attempt_2)

intervals = extract_data(infotree, body)

In [39]:
for interval in intervals:
    print("goalsBefore:")
    print("\n".join(goal for goal in interval["goalsBefore"]))
    print("-" * 3)
    print("tactic:")
    print(interval["tactic"])
    print("-" * 3)
    print("goalsAfter:")
    print("\n".join(goal for goal in interval["goalsAfter"]))
    print("-" * 20)

goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
⊢ a ^ 2 / (b + c) + b ^ 2 / (a + c) + 16 * c ^ 2 / (a + b) ≥ 1 / 9 * (64 * c - a - b)
---
tactic:
by
  have h₁ : 0 < a + b + c :=
---
goalsAfter:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
⊢ 0 < a + b + c
--------------------
goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
⊢ 0 < a + b + c
---
tactic:
 by linarith
---
goalsAfter:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ a ^ 2 / (b + c) + b ^ 2 / (a + c) + 16 * c ^ 2 / (a + b) ≥ 1 / 9 * (64 * c - a - b)
--------------------
goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ a ^ 2 / (b + c) + b ^ 2 / (a + c) + 16 * c ^ 2 / (a + b) ≥ 1 / 9 * (64 * c - a - b)
---
tactic:

  have h₂ : 0 < a * b :=
---
goalsAfter:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ 0 < a * b
--------------------
goalsBefore:
a b c : ℝ
ha : 0 < a
hb : 0 < b
hc : 0 < c
h₁ : 0 < a + b + c
⊢ 0 < a * b
---
tactic:
 by positivity
---
goalsAft